In [ ]:
import pandas as pd
import numpy as np
import pmdarima as pm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima.utils import ndiffs
from pmdarima.arima import auto_arima
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.varmax import VARMAX


In [ ]:
amazon_data = pd.read_csv('data_feed/AMZN.csv')

amazon = amazon_data[["Adj Close"]]


#### Prove casuali per il test della stazionarietà

In [ ]:
pm.plot_acf(amazon)

In [ ]:
# Enforcing stationarity - auto_arima - pmdarima

results = auto_arima(amazon, test='adf', trace=True, n_jobs=-1)
print(results)

#### Prova un pò più seria della stazionarietà

Dopo si prova a fare VARMAX.

Questo codice esegue il test di stazionarietà utilizzando il test ADF (Augmented Dickey-Fuller). Il grafico mostra la serie temporale originale, la rolling mean (media mobile) e la rolling standard deviation (deviazione standard mobile). I risultati del test ADF vengono stampati, inclusi il valore dell'ADF Statistic, il p-value e i valori critici.

In [ ]:
def test_stationarity(timeseries):
    # Calcola il rolling mean e la rolling standard deviation
    rolling_mean = timeseries.rolling(window=12).mean()
    rolling_std = timeseries.rolling(window=12).std()

    # Plotta la serie temporale originale, la rolling mean e la rolling standard deviation
    plt.plot(timeseries, color='blue', label='Original')
    plt.plot(rolling_mean, color='red', label='Rolling Mean')
    plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

    # Esegue il test ADF (Augmented Dickey-Fuller)
    result = adfuller(timeseries)

    # Stampa i risultati del test ADF
    print('Results of Augmented Dickey-Fuller Test:')
    print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

# Utilizzo del test di stazionarietà
test_stationarity(amazon)


Questa funzione make_stationary() prende la serie temporale originale come input e restituisce la serie temporale differenziata (o trasformata) che è resa stazionaria. Nell'esempio, viene applicata la differenziazione utilizzando il metodo diff(), ma puoi anche esplorare altre tecniche come trasformazioni logaritmiche o rimozione delle componenti stagionali.

In [ ]:
def make_stationary(series):
    # Calcola il numero consigliato di differenziazioni
    n_diffs = ndiffs(series, test='adf')
    print('Recommended number of differences: {}'.format(n_diffs))

    # Differenzia la serie temporale
    differenced_series = series.diff(n_diffs).dropna()

    return n_diffs, differenced_series

differenced_series = make_stationary(amazon)


In [ ]:
train_data, test_data = train_test_split(differenced_series, test_size = 0.2)

In [ ]:
# Step 4: VARMAX Model
best_aic = np.inf
best_order = (0,0)

for p in range(3):  # Specify the range of p values
    for q in range(3):  # Specify the range of q values
        model = VARMAX(train_data, order=(p, q))  # Use d=0 since we already differenced the data
        results = model.fit()
        aic = results.aic
        print(aic)
        if aic < best_aic:
            best_aic = aic
            best_order = (p, q)

print(best_order)

# Fit the VARMAX model with the best order
model = VARMAX(train_data, order=best_order)
results = model.fit()

# Step 5: Forecasting and Evaluation
n_forecast = len(test_data)
forecast = results.forecast(steps=n_forecast)  # Generate forecasts for the test set

# Evaluate the forecasts using a metric like mean squared error (MSE)
mse = mean_squared_error(test_data, forecast)
print("Mean Squared Error (MSE):", mse)

# Visualize the actual values and the forecasts
plt.plot(test_data, label='Actual')
plt.plot(forecast, label='Forecast')
plt.legend()
plt.show()

In [ ]:


def evaluate_varmax(differenced_series, p, q):
    model = VARMAX(differenced_series, order=(p, q))
    results = model.fit()
    return results

# Create a VARMAX model, p: autoregressive component, q: order of the moving average component
model = VARMAX(differenced_series, order=(p, q))

# Fit the model
results = model.fit()

# Make predictions, n: number of steps ahead for which you want to generate forecasts
n = 10
forecast = results.forecast(steps=n)

# If you want to invert the differencing and obtain predictions on the original scale:
# Assuming you have the original time series 'original_series'
predicted_values = differenced_series.shift(1).cumsum() + forecast

# Print the predicted values
print(predicted_values)

In [ ]:
import matplotlib.pyplot as plt

time_series = [10, 12, 14, 15, 18, 20, 22, 25, 28, 30]
predictions = [18, 20, 22, 25, 28]
start_index = 5

plt.plot(range(len(time_series)), time_series, label='Time Series')
plt.plot(range(start_index, start_index + len(predictions)), predictions, label='Predictions')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series with Predictions')
plt.legend()

plt.show()